In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Preprocessing

Contiguous

In [1]:
%run preprocessing ../data/SRAD2018/ 1,2 ../data False contiguous --nt 10

Processing subdirectory: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:54<00:00, 92.10it/s]
2it [02:24, 72.45s/it]

In [2]:
%run preprocessing ../data/SRAD2018/ 1 ../data True contiguous --nt 10

Processing subdirectory: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:02<00:00, 159.59it/s]
1it [01:03, 63.30s/it]

Skip

In [3]:
%run preprocessing ../data/SRAD2018/ 1,2 ../data False skip --nt=10 --stop=5

Processing subdirectory: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:16<00:00, 309.54it/s]
2it [01:06, 33.33s/it]

In [4]:
%run preprocessing ../data/SRAD2018/ 1 ../data True skip

Processing subdirectory: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:21<00:00, 470.59it/s]
1it [00:21, 21.50s/it]

### Data

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from predrain import *
sz = 512
nt = 10
bs = 1 # calculate this
num_gpus = 1
gpu_start = 2

ModuleNotFoundError: No module named 'bcolz'

In [3]:
from pathlib import Path

In [4]:
t = Path('.')

In [5]:
t.iterdir()

<generator object Path.iterdir at 0x7f111ec7b3b8>

In [7]:
next(_).is_dir()

False

In [ ]:
../data/SRAD2018/SRAD2018_Test_1/RAD_206482434212530/RAD_206482434212530_000.png

In [3]:
mt = ModelType.PredNetLeakyRelu

In [4]:
P = Predrain()
P.set_config(sz, nt, bs, mt, num_gpus, gpu_start, allow_growth=False)

In [8]:
P.get_data(pred_mode='skip', buffer_size=1, idx=range(1, 1 + 2))

((<tf.Tensor 'IteratorGetNext_2:0' shape=(?, 10, 512, 512, 1) dtype=float32>,
  <tf.Tensor 'zeros_2:0' shape=(1, 1) dtype=float32>),
 (<tf.Tensor 'IteratorGetNext_3:0' shape=(?, 10, 512, 512, 1) dtype=float32>,
  <tf.Tensor 'zeros_3:0' shape=(1, 1) dtype=float32>),
 45000,
 5000)

In [11]:
P.get_model(mt)

sz=512 trained from scratch

In [ ]:
P.train(1, max_lr=1e-2/2)

Epoch 1/1
12241/45000 [=======>......................] - ETA: 10:15:23 - loss: 0.0488

In [ ]:
lrfinder = P.lr_find(1e-2)

upsample

In [5]:
sz = 501
bs = 6
P.has_model = False

In [37]:
P.load(mt, 128, 1)

# Run script

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [10]:
%run preprocessing ../data/SRAD2018/ 1,2,3,4,5 ../data False skip\
    --nt=5 --stop=5

Processing subdirectory: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [02:45<00:00, 30.15it/s]
5it [11:34, 138.91s/it]

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
%run train --sz=256 --nt=5 --bs=7 --num_gpus=2 --gpu_start=2 --mt-idx=3\
    --pred-mode=skip --data-idx=1,2,3,4,5 --epochs=2 --max_lr=1e-3 --train\
    --comment=dual_gpu

Epoch 1/2
